In [2]:
#libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math

import random
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import max_error
from sklearn.feature_selection import SelectFromModel

In [3]:
from itertools import repeat
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [1]:
#load the data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data=pd.read_csv(r'/content/drive/MyDrive/dataset - 2020-09-24.csv')


In [5]:
data

,Name,Jersey Number,Club,Position,Nationality,Age,Appearances,Wins,Losses,Goals,...,Punches,High Claims,Catches,Sweeper clearances,Throw outs,Goal Kicks,Yellow cards,Red cards,Fouls,Offsides
0,Bernd Leno,1.0,Arsenal,Goalkeeper,Germany,28.0,64,28,16,0,...,34.0,26.0,17.0,28.0,375.0,489.0,2,0,0,NaN
1,Matt Macey,33.0,Arsenal,Goalkeeper,England,26.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN
2,Rúnar Alex Rúnarsson,13.0,Arsenal,Goalkeeper,Iceland,25.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN
3,Héctor Bellerín,2.0,Arsenal,Defender,Spain,25.0,160,90,37,7,...,NaN,NaN,NaN,NaN,NaN,NaN,23,0,125,8.0
4,Kieran Tierney,3.0,Arsenal,Defender,Scotland,23.0,16,7,5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,2,0,9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,Pedro Neto,7.0,Wolverhampton-Wanderers,Forward,Portugal,20.0,31,14,7,3,...,NaN,NaN,NaN,NaN,NaN,NaN,3,0,16,5.0
567,Raúl Jiménez,9.0,Wolverhampton-Wanderers,Forward,Mexico,29.0,78,32,23,32,...,NaN,NaN,NaN,NaN,NaN,NaN,7,0,81,52.0
568,Adama Traoré,37.0,Wolverhampton-Wanderers,Forward,Spain,24.0,105,27,47,5,...,NaN,NaN,NaN,NaN,NaN,NaN,4,0,80,13.0
569,Fábio Silva,17.0,Wolverhampton-Wanderers,Forward,Portugal,18.0,1,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 571 entries, 0 to 570
Data columns (total 59 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    571 non-null    object 
 1   Jersey Number           563 non-null    float64
 2   Club                    571 non-null    object 
 3   Position                571 non-null    object 
 4   Nationality             570 non-null    object 
 5   Age                     570 non-null    float64
 6   Appearances             571 non-null    int64  
 7   Wins                    571 non-null    int64  
 8   Losses                  571 non-null    int64  
 9   Goals                   571 non-null    int64  
 10  Goals per match         309 non-null    float64
 11  Headed goals            502 non-null    float64
 12  Goals with right foot   502 non-null    float64
 13  Goals with left foot    502 non-null    float64
 14  Penalties scored        309 non-null    fl

In [8]:
# Remove entries which do not have age, jersey number and nationality
data = data[data['Nationality'].notna()]
data = data[data['Age'].notna()]
data = data[data['Jersey Number'].notna()]

# cleaning the percentage sign
data['Cross accuracy %'] = data['Cross accuracy %'].str.replace(r'%', '').astype(float)
data['Shooting accuracy %'] = data['Shooting accuracy %'].str.replace(r'%', '').astype(float)
data['Tackle success %'] = data['Tackle success %'].str.replace(r'%', '').astype(float)

features = data.columns
data_clean = data[features]
data_clean.head()

data_clean_appNonZero = data_clean[data_clean['Appearances'] > 0] #prevents division by zero for players who are yet to make appearances

# take care off the data type for division
# separate cols with dtype objects and cols that may not be divided by appearances

cols = features.drop(['Age', 'Name', 'Appearances', 'Club', 'Nationality', 'Jersey Number', 'Cross accuracy %', 'Position', 'Goals per match',
                      'Passes per match','Tackle success %', 'Shooting accuracy %'])
data_clean_appNonZero.loc[:, cols] = data_clean_appNonZero.loc[:, cols].div(data_clean_appNonZero['Appearances'], axis=0)

# positional classifications on the data as is
goalies = data[data['Position'] == 'Goalkeeper']
defenders = data[data['Position'] == 'Defender']
midfielders = data[data['Position'] == 'Midfielder']
forwards = data[data['Position'] == 'Forward']

# palyers who have made atleast 38 games (a seasons worth games)
# data as is
data_38app = data[data['Appearances'] >=38]
goalies_38app = goalies[goalies['Appearances'] >= 38]
defenders_38app = defenders[defenders['Appearances'] >= 38]
midfilders_38app = midfielders[midfielders['Appearances'] >= 38]
forwards_38app = forwards[forwards['Appearances'] >= 38]

# palyers who have made atleast 38 games (a seasons worth games)
# data normalized
all_players = data_clean_appNonZero[data_clean_appNonZero['Appearances'] >= 38]
goalies_ = data_clean_appNonZero[(data_clean_appNonZero['Position'] == 'Goalkeeper') & (data_clean_appNonZero['Appearances'] >= 38)]
defenders_ = data_clean_appNonZero[(data_clean_appNonZero['Position'] == 'Defender') & (data_clean_appNonZero['Appearances'] >= 38)]
midfielders_ = data_clean_appNonZero[(data_clean_appNonZero['Position'] == 'Midfielder') & (data_clean_appNonZero['Appearances'] >= 38)]
forwards_ = data_clean_appNonZero[(data_clean_appNonZero['Position'] == 'Forward') & (data_clean_appNonZero['Appearances'] >= 38)]

In [9]:
data

,Name,Jersey Number,Club,Position,Nationality,Age,Appearances,Wins,Losses,Goals,...,Punches,High Claims,Catches,Sweeper clearances,Throw outs,Goal Kicks,Yellow cards,Red cards,Fouls,Offsides
0,Bernd Leno,1.0,Arsenal,Goalkeeper,Germany,28.0,64,28,16,0,...,34.0,26.0,17.0,28.0,375.0,489.0,2,0,0,NaN
1,Matt Macey,33.0,Arsenal,Goalkeeper,England,26.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN
2,Rúnar Alex Rúnarsson,13.0,Arsenal,Goalkeeper,Iceland,25.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,NaN
3,Héctor Bellerín,2.0,Arsenal,Defender,Spain,25.0,160,90,37,7,...,NaN,NaN,NaN,NaN,NaN,NaN,23,0,125,8.0
4,Kieran Tierney,3.0,Arsenal,Defender,Scotland,23.0,16,7,5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,2,0,9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,Pedro Neto,7.0,Wolverhampton-Wanderers,Forward,Portugal,20.0,31,14,7,3,...,NaN,NaN,NaN,NaN,NaN,NaN,3,0,16,5.0
567,Raúl Jiménez,9.0,Wolverhampton-Wanderers,Forward,Mexico,29.0,78,32,23,32,...,NaN,NaN,NaN,NaN,NaN,NaN,7,0,81,52.0
568,Adama Traoré,37.0,Wolverhampton-Wanderers,Forward,Spain,24.0,105,27,47,5,...,NaN,NaN,NaN,NaN,NaN,NaN,4,0,80,13.0
569,Fábio Silva,17.0,Wolverhampton-Wanderers,Forward,Portugal,18.0,1,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.0


In [29]:
import plotly.io as pio
pio.renderers.default = 'colab'


In [30]:
df = data
fig = px.pie(df,
             values='Appearances',
             names='Nationality',
             title='Countries represented in the EPL by number of appearances',
             )
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(title_text='<b> Nr. of appearances per country <b>',
                  title_x=0.5,
                  titlefont=dict(color='black',
                            size=28,
                            family="Courier New, monospace",),
                  width=600,
                  height=600,
                  showlegend=False,
                 )
iplot(fig)

In [31]:
df = data
fig = px.sunburst(df,
                   path=['Position', 'Nationality'],
                   values='Appearances',
                 )
fig.update_layout(title_text='<b>Players position by country <b>',
                  title_x=0.5,
                  titlefont=dict(color='black',
                            size=28,
                            family="Courier New, monospace",),
                  width=600,
                  height=600,
                  showlegend=False,
                 )
iplot(fig)

In [32]:
df = data
fig = px.bar(df, x="Position", y="Appearances",color='Club',
             hover_data=["Name"],
             width=750, height=600,)
fig.update_layout(
             template='ggplot2',
             title='<b>Players appearance by position<b>',
             titlefont={'size':24})
iplot(fig)

In [33]:
fig = px.bar(df, y="Club", x="Appearances",color='Position',
             hover_data=["Name"],
             width=750,
             height=600,
             )
fig.update_layout(
             template='ggplot2',
             title='<b>Appearance by club<b>',
             titlefont={'size':24},
)
iplot(fig)

In [34]:
df = data
age_avg=df['Age'].mean()
fig = px.violin(df, y="Age", x="Position", box=True,
                title='<b> Players Age distribution by position (avg. age dotted line)<b>',
                width=600,height=400,template='simple_white')
fig.add_shape(
    type="line", line_color="blue", line_width=3, opacity=1, line_dash="dot",
    x0=0, x1=1, xref="paper", y0=age_avg, y1=age_avg, yref="y"
)
iplot(fig)

fig = px.box(df, y="Club", x="Age",
            title='<b>Players Age distribution by club (avg. age dotted line)<b>',
            width=750,height=750,template='ggplot2')
fig.add_shape(
    type="line", line_color="black", line_width=3, opacity=1, line_dash="dot",
    y0=0, y1=1, yref="paper", x0=age_avg, x1=age_avg, xref="x"
)
# fig.update_layout( yaxis={'categoryorder':'total descending'})
iplot(fig)

In [35]:
from plotly.subplots import make_subplots

head = 5
df1=goalies_38app.sort_values(by='Clean sheets', ascending=False).head(head)
df2=goalies_38app.sort_values(by='Saves', ascending=False).head(head)
df3=goalies_38app.sort_values(by='High Claims', ascending=False).head(head)
df4=goalies_38app.sort_values(by='Catches', ascending=False).head(head)

df11=goalies_.sort_values(by='Clean sheets', ascending=False).head(head)
df12=goalies_.sort_values(by='Saves', ascending=False).head(head)
df13=goalies_.sort_values(by='High Claims', ascending=False).head(head)
df14=goalies_.sort_values(by='Catches', ascending=False).head(head)

fig = make_subplots(
    rows=4, cols=2,
    subplot_titles=('Clean sheets (overall)', 'Clean sheets (per-game)','Saves (overall)','Saves (per-game)',
                    'High Claims (overall)', 'High Claims (per-game)','Catches (overall)', 'Catches (per-game)'),
    horizontal_spacing = 0.12,
    vertical_spacing = 0.075)

fig.add_trace(go.Bar(
                y=df1["Name"],
                x=df1['Clean sheets'],
                hovertext=df1['Club'],
                orientation='h'),
                row=1, col=1)

fig.add_trace(go.Bar(
                y=df2["Name"],
                x=df2['Saves'],
                hovertext=df2['Club'],
                orientation='h'),
                row=2, col=1)

fig.add_trace(go.Bar(
                y=df3["Name"],
                x=df3['High Claims'],
                hovertext=df3['Club'],
                orientation='h'),
                row=3, col=1)

fig.add_trace(go.Bar(
                y=df4["Name"],
                x=df4['Catches'],
                hovertext=df4['Club'],
                orientation='h'),
                row=4, col=1)


fig.add_trace(go.Bar(
                y=df11["Name"],
                x=df11['Clean sheets'],
                hovertext=df11['Club'],
                orientation='h'),
                row=1, col=2)

fig.add_trace(go.Bar(
                y=df12["Name"],
                x=df12['Saves'],
                hovertext=df12['Club'],
                orientation='h'),
                row=2, col=2)

fig.add_trace(go.Bar(
                y=df13["Name"],
                x=df13['High Claims'],
                hovertext=df13['Club'],
                orientation='h'),
                row=3, col=2)

fig.add_trace(go.Bar(
                y=df14["Name"],
                x=df14['Catches'],
                hovertext=df14['Club'],
                orientation='h'),
                row=4, col=2)

fig.update_traces(marker_color= ['rgb(110,102,250)','rgb(210,202,82)','rgb(210,202,82)','rgb(210,202,82)',
                                 'rgb(210,202,82)',], marker_line_color='rgb(8,48,107)',
                  marker_line_width=2.5, opacity=0.6)
fig.update_layout(title_text='<b> Top goalkeepers stat<b>',
                  titlefont={'size':28},
                  title_x=0.5,
                  showlegend=False,
                  autosize=False,
                  width=1300,
                  height=1200,
                  template='ggplot2',
                  paper_bgcolor='lightgray',
                  #plot_bgcolor='lightgray',
                 )
fig.show()

In [36]:
defenders_attr =['Blocked shots', 'Interceptions', 'Clearances','Headed Clearance', 'Clearances off line',
                 'Duels won','Successful 50/50s', 'Aerial battles won']
# top=5
# defenders_attr =['Tackles', 'Tackle success'Interceptions' %', 'Last man tackles', 'Blocked shots', 'Interceptions', 'Clearances',
#                  'Headed Clearance', 'Clearances off line', 'Recoveries', 'Duels won','Successful 50/50s', 'Aerial battles won']
# for atr in defenders_attr:
#     text = 5
#     df = data_38app[data_38app["Position"] == 'Defender'].sort_values(by=atr, ascending=False).head(top)
#     fig = px.bar(df, x="Name",
#                  y=atr,
#                  color='Club',
#                  hover_name=None,
#                  title="Defender defensive ability: Top {} {} ".format(text, atr.lower()))
#     fig.update_layout(autosize=False, width=1000, height=500)
#     iplot(fig)


top = 5
fig = make_subplots(
    rows=5, cols=2,
    horizontal_spacing = 0.05,
    vertical_spacing = 0.075,
    subplot_titles=('Blocked shots (overall)', 'Blocked shots (per-game)','Interceptions (overall)', 'Interceptions (per-game)','Clearances (overall)',
                    'Clearances (per-game)','Headed Clearance (overall)', 'Headed Clearance (per-game)','Clearances off line (overall)', 'Clearances off line (per-game)'),
    )


df = defenders_38app.sort_values(by='Blocked shots', ascending=False).head(top)
fig.add_trace(go.Bar(x=df["Name"],
             y=df['Blocked shots'],
             #color='Club',
             #hover_name=None,
             orientation='v'),
             row=1, col=1)

df = defenders_38app.sort_values(by='Interceptions', ascending=False).head(top)
fig.add_trace(go.Bar(x=df["Name"],
             y=df['Interceptions'],
             #color='Club',
             #hover_name=None,
             orientation='v'),
             row=2, col=1)

df = defenders_38app.sort_values(by='Clearances', ascending=False).head(top)
fig.add_trace(go.Bar(x=df["Name"],
             y=df['Clearances'],
             #color='Club',
            #hover_name=None,
             orientation='v'),
             row=3, col=1)


df = defenders_38app.sort_values(by='Headed Clearance', ascending=False).head(top)
fig.add_trace(go.Bar(x=df["Name"],
             y=df['Headed Clearance'],
             #color='Club',
             #hover_name=None,
             orientation='v'),
             row=4,col=1)

df = defenders_38app.sort_values(by='Clearances off line', ascending=False).head(top)
fig.add_trace(go.Bar( x=df["Name"],
             y=df['Clearances off line'],
             #color='Club',
             #hover_name=None,
             orientation='v'),
             row=5, col=1)


df = defenders_.sort_values(by='Blocked shots', ascending=False).head(top)
fig.add_trace(go.Bar( x=df["Name"],
             y=df['Blocked shots'],
             #color='Club',
             #hover_name=None,
             orientation='v'),
             row=1, col=2)

df = defenders_.sort_values(by='Interceptions', ascending=False).head(top)
fig.add_trace(go.Bar( x=df["Name"],
             y=df['Interceptions'],
             #color='Club',
             #hover_name=None,
             orientation='v'),
             row=2, col=2)


df = defenders_.sort_values(by='Clearances', ascending=False).head(top)
fig.add_trace(go.Bar(x=df["Name"],
             y=df['Clearances'],
             #color='Club',
             #hover_name=None,
             orientation='v'),
             row=3, col=2)


df = defenders_.sort_values(by='Headed Clearance', ascending=False).head(top)
fig.add_trace(go.Bar( x=df["Name"],
             y=df['Headed Clearance'],
             #color='Club',
             #hover_name=None,
             orientation='v'),
             row=4,col=2)

df = defenders_.sort_values(by='Clearances off line', ascending=False).head(top)
fig.add_trace(go.Bar( x=df["Name"],
             y=df['Clearances off line'],
             #color='Club',
             #hover_name=None,
             orientation='v'),
             row=5, col=2)

#fig.update_layout(title_text='Top Defender Qualities', title_x=0.5)
fig.update_traces(marker_color= ['rgb(96, 96, 96)','rgb(210,202,82)','rgb(210,202,82)','rgb(210,202,82)',
                                 'rgb(210,202,82)', 'rgb(210,202,82)'], marker_line_color='rgb(8,48,107)',
                  marker_line_width=2.5, opacity=0.6)

fig.update_layout(title_text='<b>Top Defender Qualities<b>',
                  titlefont={'size':28},
                  title_x=0.5,
                  showlegend=False,
                  autosize=False,
                  width=1300,
                  height=1300,
                  template='ggplot2',
                  paper_bgcolor='lightgray')
fig.show()

In [37]:
mid_field_attr_D =['Recoveries','Duels won','Successful 50/50s','Aerial battles won']

col=2
row=4

top = 5
fig = make_subplots(
    rows=4, cols=2,
    subplot_titles=('Recoveries (overall)','Recoveries (per-game)','Duels won (overall)','Duels won (per-game)','Successful 50/50s (overall)',
                    'Successful 50/50s (per-game)','Aerial battles won (overall)', 'Aerial battles won (per-game)'))

for i, atr in enumerate(mid_field_attr_D):
    df = data_38app[data_38app["Position"] == 'Midfielder'].sort_values(by=atr, ascending=False).head(top)
    fig.add_trace(go.Bar(x=df['Name'],
         y=df[atr],
         orientation='v'),
         row=i+1, col=1)

for j, atr in enumerate(mid_field_attr_D):
    df = midfielders_.sort_values(by=atr, ascending=False).head(top)
    fig.add_trace(go.Bar(x=df['Name'],
         y=df[atr],
         orientation='v'),
         row=j+1, col=2)

    fig.update_traces(marker_color= ['rgb(96, 96, 96)','rgb(110,202,82)','rgb(110,202,82)','rgb(110,202,82)',
                                    'rgb(110,202,82)', 'rgb(110,202,82)'], marker_line_color='rgb(8,48,107)',
                  marker_line_width=2.5, opacity=0.6)

fig.update_layout(title_text='<b>Top Midfield Qualities: Defense Ability<b>',
                  titlefont={'size': 28, 'family':'Courier'},
                  title_x=0.5,
                  showlegend=False,
                  autosize=False,
                  width=1200,
                  height=1200,
                  template='ggplot2',
                  paper_bgcolor='lightgray')
iplot(fig)

In [38]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'


head = 10
table_header = ['Rank', 'Total goals','Goals with right foot','Goals with left foot','Headed goals']
df = data_38app

fig = go.Figure(data=[go.Table(

    header=dict(values=list(table_header),
                    line_color='darkslategray',
                    fill_color=headerColor,
                    align=['left'],
                    font=dict(color='white', size=20),
                    height=30
                               ),

    cells=dict(values=[list(np.arange(1, head+1)),
        data_38app.sort_values(by='Goals', ascending=False)['Name'].head(head),

        data_38app.sort_values(by='Goals with right foot', ascending=False)['Name'].head(head),
        data_38app.sort_values(by='Goals with left foot', ascending=False)['Name'].head(head),
        data_38app.sort_values(by='Headed goals', ascending=False)['Name'].head(head)],

       fill_color=[[rowOddColor,rowEvenColor]*5],
       font=dict(color='black', size=16, family="Courier New, monospace",),
       align='left', height=25,)
        )
])
fig.update_layout(title_text='TOP {} GOAL SCORERS'.format(head), title_x=0.5, font=dict(color='white', size=20, family="Courier New, monospace",))
fig.update_layout(width=1200, height=550, template='plotly_dark')
fig.show()

In [39]:
#data = data_38app.head(10)
fig = px.pie(data,
             values='Goals',
             names='Nationality',
             title='<b>Player of which country score the most goals? <b>',
             width=550, height=550,
             )
fig.update_traces(textposition='inside',
                  textinfo='percent+label',
                  showlegend= False,
                 )
iplot(fig)

In [40]:
#Headed goals
hg_f =data[data['Position'] == 'Forward']['Headed goals'].sum()
hg_m =data[data['Position'] == 'Midfielder']['Headed goals'].sum()
hg_d =data[data['Position'] == 'Defender']['Headed goals'].sum()
hg_kg =data[data['Position'] == 'Goalkeeper']['Headed goals'].sum()
#Goals with right foot
rfg_f =data[data['Position'] == 'Forward']['Goals with right foot'].sum()
rfg_m =data[data['Position'] == 'Midfielder']['Goals with right foot'].sum()
rfg_d =data[data['Position'] == 'Defender']['Goals with right foot'].sum()
rfg_kg =data[data['Position'] == 'Goalkeeper']['Goals with right foot'].sum()
#Goals with left foot
lfg_f =data[data['Position'] == 'Forward']['Goals with left foot'].sum()
lfg_m =data[data['Position'] == 'Midfielder']['Goals with left foot'].sum()
lfg_d =data[data['Position'] == 'Defender']['Goals with left foot'].sum()
lfg_kg =data[data['Position'] == 'Goalkeeper']['Goals with left foot'].sum()

In [41]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=data['Club'],
    y=data['Goals with right foot'],
    name='Right foot goals',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=data['Club'],
    y=data['Goals with left foot'],
    name='Left foot goals',
   marker_color='lightsalmon'
))

fig.add_trace(go.Bar(
    x=data['Club'],
    y=data['Headed goals'],
    name='Headers',
   marker_color='lightseagreen'
))

fig.update_layout(barmode='group',)# xaxis_tickangle=-45)
fig.update_layout(title_text="<b>Goal distribution by club<b>",
                  titlefont={'size': 24, 'family': 'Courier'},
                  width=750,
                  height =500,
                  template='simple_white'
                 )

iplot(fig)


In [42]:
df = data

fig = px.sunburst(df, path=['Position', 'Club'], values='Headed goals')
fig.update_layout(title_text="<b>Headed goals (Club, position)<b>",
                  titlefont= {'size': 24},
                  width=500,
                  height=500)
iplot(fig)

fig1 = px.sunburst(df, path=['Position', 'Club'], values='Goals with left foot')
fig1.update_layout(title_text="<b>Left footed goals (Club, position<b>",
                   titlefont= {'size': 24,},
                   width=500,
                   height=500)
iplot(fig1)

fig2 = px.sunburst(df, path=['Position', 'Club'], values='Goals with right foot')
fig2.update_layout(title_text="<b>Right footed goals (Club, position)<b>",
                   titlefont= {'size': 24,},
                   width=500,
                   height=500)
iplot(fig2)

In [43]:
goals_groupedby_clubs=data.groupby(['Club']).agg({'Goals with left foot':sum,
                                                  'Goals with right foot':sum, 'Headed goals': sum })


all_nodes = ['Forwards', 'Midfielders', 'Defenders', 'Right foot goals', 'Left foot goals', 'Headed goals',
           'Arsenal', 'Aston-Villa', 'Brighton-and-Hove-Albion', 'Burnley',
           'Chelsea', 'Crystal-Palace', 'Everton', 'Fulham', 'Leeds-United',
           'Leicester-City', 'Liverpool', 'Manchester-City', 'Manchester-United',
           'Newcastle-United', 'Sheffield-United', 'Southampton',
           'Tottenham-Hotspur', 'West-Bromwich-Albion', 'West-Ham-United',
           'Wolverhampton-Wanderers']


# source nodes
list_1 = [0,1,2]
list_2 = [3,4,5]
source = 3*list_1 + 20*list_2


# target nodes
target = []
for tar in range(3, 26):
    target.extend(repeat(tar, 3))

#
R = goals_groupedby_clubs['Goals with right foot'].values
L = goals_groupedby_clubs['Goals with left foot'].values
H = goals_groupedby_clubs['Headed goals'].values

#
goals_scored = []
for i in range(len(R)):
    goals_scored.append(R[i])
    goals_scored.append(L[i])
    goals_scored.append(H[i])

value = [rfg_f, rfg_m, rfg_d, lfg_f, lfg_m, lfg_d, hg_f, hg_m, hg_d] + goals_scored


fig = go.Figure(
      data=[go.Sankey(
      node = dict(
      pad = 2,
      thickness = 75,
      line = dict(color = "gray", width = 0.75),
      label = all_nodes,
      color = ['#67AEE1', '#ff6e4a', '#48bf91'] +
          3*['gold'] + 20*['#babad4']#node_colors

    ),

    link = dict(
       source = source,

       target = target,

       value =  value,

       color = 26*['#94C6EA', '#ffb6a4', '#a3dfc8'],
       #color = 26*['#d8d8d8', '#a7a7a7', '#4f4f4f']
))])

fig.update_layout(title_text="<b> Distribution of goals: <b> "\
                  '<br><span style="font-size:16px; color: darkgray"> By player position, club and part-of-body scored by',
                  titlefont={'size': 28, 'family': 'Courier'})
fig.update_traces(textfont_family='Courier', selector=dict(type='sankey'))
iplot(fig)


In [44]:
unwanted_records = ['Losses', 'Big chances missed','Own goals','Errors leading to goal', 'Red cards']
col=2
row=5

top = 5
fig = make_subplots(
    rows=5, cols=2,
    subplot_titles=('Losses (overall)', 'Losses (per-game)','Big chances missed (overall)','Big chances missed (per-game)',
                    'Own goals (overall)','Own goals (per-game)', 'Errors leading to goal (overall)',
                    'Errors leading to goal (per-game)','Red cards (overall)', 'Red cards (per-game)'))

for i, atr in enumerate(unwanted_records):
    df = data_38app.sort_values(atr, ascending=False).head(top)
    fig.add_trace(go.Bar(x=df['Name'],
         y=-df[atr],
         orientation='v'),
         row=i+1, col=1)

for j, atr in enumerate(unwanted_records):
    df = all_players.sort_values(atr, ascending=False).head(top)
    fig.add_trace(go.Bar(x=df['Name'],
         y=-df[atr],
         orientation='v'),
         row=j+1, col=2)

    fig.update_traces(marker_color= ['rgb(255, 0, 0)'],
                  marker_line_width=2.5, opacity=0.6)

fig.update_layout(title_text='<b> List of the unfortunates <b>',
                  titlefont={'size': 28, 'family':'Courier'},
                  showlegend=False,
                  autosize=False,
                  width=1200, height=1100,
                  paper_bgcolor='lightgray',
                  plot_bgcolor='lightgray',
                 )
iplot(fig)